# 挖掘建模

经过数据探索和数据预处理，得到了可以直接建模的数据。根据数据挖掘目标和数据形式可以建立分类与预测、聚类分析、关联分析、时序模式和偏差检测等模型，帮助企业提取数据中蕴含的商业价值，提高企业的竞争力。

## 分类与预测

分类和预测是预测问题的两种主要类型
- 分类主要是预测分类标号
- 预测主要是建立连续值函数模型，预测给定自变量对应的因变量的值

### 常见的分类与预测算法

- 回归分析
    - 线性回归
    - 非线性回归
    - Logistic回归
    - 岭回归
    - 主成分回归
    - 偏最小二乘回归
- 决策树
- 人工神经网络
- 贝叶斯网络
- 支持向量机

### 回归分析

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RandomizedLogisticRegression

In [22]:
# 导入数据集
filename = './data/data_5/bankloan.xls'
data = pd.read_excel(filename)

# 准备数据
X = data.drop('违约', axis=1).values
y = data['违约'].values

In [23]:
# 无特征选择的建模
lr = LogisticRegression(solver='liblinear').fit(X, y)
print('模型的平均正确率为：{}'.format(lr.score(X, y)))

模型的平均正确率为：0.8057142857142857


In [24]:
# 特征选择
features = data.columns[:-1].copy()
rlr = RandomizedLogisticRegression().fit(X, y)
print('特征得分为：{}'.format(rlr.scores_))
print('特征选择结果为：{}'.format(rlr.get_support()))
print('选择的有效特征为： {}'.format(features[rlr.get_support()].values))
X = data[features[rlr.get_support()]].values
print('特征选择后数据集的形状为：{}'.format(X.shape))

特征得分为：[0.095 0.105 0.99  0.36  0.    1.    0.615 0.035]
特征选择结果为：[False False  True  True False  True  True False]
选择的有效特征为： ['工龄' '地址' '负债率' '信用卡负债']
特征选择后数据集的形状为：(700, 4)


In [25]:
# 特征选择后的建模
lr = LogisticRegression().fit(X, y)
print('模型的平均正确率为：{}'.format(lr.score(X, y)))

模型的平均正确率为：0.8142857142857143


特征选择方法：
- 递归特征消除
- 稳定性选择

**Logistic回归本质上是一种线性模型，这里的有效性检验本质上还是一种线性相关检验，因此筛选出来的变量只是与结果具有较强的线性相关性，变量没有被保留，也只是因为没有明显的线性相关性，但是可能存在一些非线性的关系，实际操作中需要根据问题的背景对筛选结果进行分析。**

对于非线性关系的变量筛选方法有决策树、神经网络等。

### 决策树